In [1]:
import pandas as pd
import numpy as np

def rmsle(y_pred, y_test) : 
    assert len(y_test) == len(y_pred)
    return np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_test))**2))

In [2]:
train = pd.read_csv('./data/transformed_train.csv')
train = train.set_index('Id')

test = pd.read_csv('./data/transformed_test.csv')
test = test.set_index('Id')

In [3]:
pd.options.display.max_columns = 150
test.head(10)

,MSSubClass,LotFrontage,LotArea,LotShape,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,_1stFlrSF,_2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,_3SsnPorch,ScreenPorch,PoolArea,PoolQC,MiscVal,YrSold,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,MSZoning_nan,Street_Pave,Street_nan,Alley_Pave,Alley_nan,LandContour_HLS,LandContour_Low,LandContour_Lvl,LandContour_nan,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LotConfig_nan,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,...,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,Exterior2nd_nan,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,MasVnrType_nan,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Foundation_nan,Heating_GasW,Heating_Grav,Heating_Wall,Heating_nan,CentralAir_Y,CentralAir_nan,Electrical_FuseF,Electrical_FuseP,Electrical_SBrkr,Electrical_nan,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_nan,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_nan,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_nan,MoSold_Aug,MoSold_Dec,MoSold_Feb,MoSold_Jan,MoSold_Jul,MoSold_Jun,MoSold_Mar,MoSold_May,MoSold_Nov,MoSold_Oct,MoSold_Sep,MoSold_nan,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleType_nan,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_nan
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1461,20,80,11622,4,3,5,6,1961,1961,0,3,3,3,3,1,3,468,2,144,270,882,3,896,0,0,896,0,0,1,0,2,1,3,5,7,0,0,1961,1,1,730,3,3,2,140,0,0,0,120,0,0,0,2010,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
1462,20,81,14267,3,3,6,6,1958,1958,108,3,3,3,3,1,5,923,1,0,406,1329,3,1329,0,0,1329,0,0,1,1,3,1,4,6,7,0,0,1958,1,1,312,3,3,2,393,36,0,0,0,0,0,12500,2010,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
1463,60,74,13830,3,3,5,5,1997,1998,0,3,3,4,3,1,6,791,1,0,137,928,4,928,701,0,1629,0,0,2,1,3,1,3,6,7,1,3,1997,3,2,482,3,3,2,212,34,0,0,0,0,0,0,2010,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
1464,60,78,9978,3,3,6,6,1998,1998,20,3,3,3,3,1,6,602,1,0,324,926,5,926,678,0,1604,0,0,2,1,3,1,4,7,7,1,4,1998,3,2,470,3,3,2,360,36,0,0,0,0,0,0,2010,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
1465,120,43,5005,3,3,8,5,1992,1992,0,4,3,4,3,1,5,263,1,0,1017,1280,5,1280,0,0,1280,0,0,2,0,2,1,4,5,7,0,0,1992,2,2,506,3,3,2,0,82,0,0,144,0,0,0,2010,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
1466,60,75,100

## Transform test dataframe so columns match train

In [4]:
missing_cols = []

for col in train.columns:
    if col not in test.columns and col != 'SalePrice':
        missing_cols.append(col)

In [5]:
for col in missing_cols:
    test[col] = 0

In [6]:
len(test.columns)

235

In [7]:
len(train.columns)

236

## Create train and test from train dataframe and fit model

In [8]:
X = train.iloc[:, train.columns != 'SalePrice']
y = train.SalePrice

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [10]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [11]:
gbm = xgb.XGBRegressor()
reg_cv = GridSearchCV(gbm, {"colsample_bytree":[1.0],"min_child_weight":[1.0,1.2]
                            ,'max_depth': [3,4,6], 'n_estimators': [500,1000]}, verbose=1, n_jobs = -1)
reg_cv.fit(X_train, y_train.apply(np.log))

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:   49.5s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'colsample_bytree': [1.0], 'min_child_weight': [1.0, 1.2], 'max_depth': [3, 4, 6], 'n_estimators': [500, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [28]:
regr = xgb.XGBRegressor()
regr.fit(X_train, y_train.apply(np.log))

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [17]:
reg_cv.best_params_

{'colsample_bytree': 1.0,
 'max_depth': 4,
 'min_child_weight': 1.0,
 'n_estimators': 500}

In [29]:
y_predict = regr.predict(X_test)

In [30]:
y_predict

array([11.955308 , 11.972266 , 12.582214 , 11.450511 , 11.502896 ,
       12.004633 , 12.188965 , 12.081382 , 11.446606 , 11.664088 ,
       11.825808 , 11.571554 , 12.416982 , 11.992837 , 11.894484 ,
       11.437569 , 11.82065  , 11.490819 , 11.802845 , 11.763729 ,
       11.849711 , 12.0788   , 11.65104  , 12.722688 , 11.888602 ,
       11.877659 , 11.678493 , 12.746041 , 12.527451 , 11.955233 ,
       11.89441  , 12.423494 , 11.634617 , 11.764498 , 11.5293   ,
       12.194814 , 12.114153 , 12.681197 , 10.852644 , 11.852862 ,
       11.739876 , 12.784623 , 12.447746 , 12.074    , 11.714735 ,
       11.819565 , 11.813535 , 11.865717 , 11.604088 , 12.436897 ,
       12.069935 , 12.272559 , 11.765506 , 12.239573 , 12.254303 ,
       11.699928 , 11.669134 , 12.230166 , 12.083535 , 11.834114 ,
       11.695265 , 12.362788 , 11.824584 , 12.136393 , 12.753455 ,
       12.091637 , 11.85241  , 12.432015 , 11.776243 , 11.418714 ,
       12.262257 , 12.029282 , 12.464547 , 11.629489 , 11.9366

In [31]:
rmsle(y_predict, y_test.apply(np.log))

0.009419616106348257

In [19]:
test = test[X_train.columns]

In [20]:
test_predict = regr.predict(test)

In [21]:
len(test_predict)

1459

In [22]:
id_price = list(zip(range(1461, 2920), np.exp(test_predict)))

In [23]:
id_price = list(map(lambda x: [x[0], round(x[1], 1)], id_price))

In [24]:
len(id_price)

1459

In [25]:
final_df = pd.DataFrame(id_price, columns=['Id', 'SalePrice'])

In [26]:
final_df

,Id,SalePrice
0,1461,119370.500000
1,1462,154759.406250
2,1463,199790.000000
3,1464,195642.406250
4,1465,181113.500000
5,1466,172150.593750
6,1467,170574.406250
7,1468,169761.000000
8,1469,177576.593750
9,1470,117947.101562


In [27]:
final_df.to_csv('./data/submit_predictions.csv', index=False)